In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')
df

## **Exploration Data Analysis** 

This dataset has 5 objects in data type, we have to convert to int or float so that machine learning can learn it well

In [ ]:
df.describe()

### **Check Missing Value**

In [ ]:
df_missing = df.isna().sum()
print(df_missing[df_missing > 0])

In [ ]:
df['smoking_status'] = df['smoking_status'].replace(['Unknown'],'NaN')

### **Check Data Type**

In [ ]:
df.info()

In [ ]:
sns.countplot(df.dtypes.map(str))

In [ ]:
categoric_cols = [categoric for categoric in df.columns if df[categoric].dtype == 'object' ]
categoric_cols

### **Univariant Analysis**

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(20, 5), sharey=True)
sns.countplot(ax=axes[0], x=df.gender)

sns.countplot(ax=axes[1], x=df.ever_married)

sns.countplot(ax=axes[2], x=df.work_type)

sns.countplot(ax=axes[3], x=df.Residence_type)

sns.countplot(ax=axes[4], x=df.smoking_status)

In [ ]:
df.drop(df.index[df['gender'] == 'Other'], inplace = True)

In [ ]:
feature = ['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi', 'stroke']

df[feature].hist(figsize=(12, 8))
plt.show()

### **Bivariant Analysis**

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(data=df.corr(), annot=True)
plt.show()

In [ ]:
scat = sns.PairGrid(df, x_vars=df.columns, y_vars=['stroke'] )
scat.map(sns.scatterplot)

In [ ]:
df

In [ ]:
df.groupby(['gender', 'ever_married'])['stroke'].sum().plot(kind='bar')

### Check By Age

## **Modeling**

In [ ]:
# preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [ ]:
# model library
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X = df.drop(columns=['id', 'stroke'])
y = df['stroke']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [ ]:
numerical_pipeline = Pipeline([
    ('scaling', StandardScaler()),
    ('impute', SimpleImputer(strategy='mean'))
])

categorical_pipeline = Pipeline([
    ('onehot', OneHotEncoder()),
    ('impute', SimpleImputer(strategy='most_frequent'))
])

preprocessing = ColumnTransformer([
    ('numeric', numerical_pipeline, ['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi']),
    ('categoric', categorical_pipeline, ['gender', 'ever_married', 'work_type', 'Residence_type',
       'smoking_status'])
])

pipeline = Pipeline([
    ('pre', preprocessing),
    ('algo', RandomForestClassifier(random_state=42))
])

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
pipeline.get_params()

In [ ]:
parameter = {
    'algo__n_estimators':[100,200, 350, 500], 
    'algo__min_samples_leaf':[2, 10, 30]
}

In [ ]:
model = GridSearchCV(pipeline, parameter, cv=3, n_jobs=-1, verbose=1)
model.fit(X_train, y_train)

In [ ]:
print(model.score(X_train, y_train)), print(model.score(X_test, y_test))

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [ ]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f_acc = f1_score(y_test, y_pred)

print(accuracy)
print(f_acc)